<a href="https://colab.research.google.com/github/neqkir/working-with-tranformers/blob/main/BERT/bertsumabs-text-summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade 
!pip install -q git+https://github.com/microsoft/nlp-recipes.git
!pip install jsonlines
!pip install pyrouge
!pip install scrapbook

import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch
import nltk
from nltk import tokenize
import pandas as pd
import pprint
import scrapbook as sb

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp import models
from utils_nlp.models import transformers 
from utils_nlp.models.transformers.abstractive_summarization_bertsum \
     import BertSumAbs, BertSumAbsProcessor
from utils_nlp.models.transformers.datasets import SummarizationDataset
from utils_nlp import eval
from utils_nlp.eval import rouge
from utils_nlp.dataset.cnndm import CNNDMSummarizationDataset
from utils_nlp.eval import compute_rouge_python